In [1]:
#! pip install jupyterlab-widgets==1.1.1 ipywidgets==7.7.2
#! pip install plotly==5.22.0

In [2]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module simple_controls
loading module save_resampled
loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

### In this example, the embeddings are generated from a Barlow Twin model trained on the Left CINGULATE region on UKB for the UKB and HCP datasets.

 -For the HCP dataset, the associated buckets can be found in: 

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Lbuckets

 -For the UKB dataset, the buckets are not generated yet, but it would be found in:

/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lbuckets

### Load your data

In [4]:
#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/hcp_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/02_Heritability_Left_PCS_HCP/Output/2024-05-13/09-33-29_206/UKB_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/01_Heritability_Right_PCS_HCP/Output/2024-05-28/10-37-46_0/HCP_random_epoch60_embeddings/full_embeddings.csv", index_col=0)
#embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/01_Heritability_Right_PCS_HCP/Output/2024-05-28/10-37-46_0/UKB_random_epoch60_embeddings/full_embeddings.csv", index_col=0)
embeddings_HCP = pd.read_csv("/neurospin/dico/adufournet/Runs/04_Heritability_Right_PCS_HCP_dim10/Output/2024-05-29/19-16-32_3/HCP_random_epoch40_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB = pd.read_csv("/neurospin/dico/adufournet/Runs/04_Heritability_Right_PCS_HCP_dim10/Output/2024-05-29/19-16-32_3/UKB_random_epoch40_embeddings/full_embeddings.csv", index_col=0)
embeddings_UKB.head()

embeddings_HCP = embeddings_HCP
embeddings_UKB = embeddings_UKB
embeddings_UKB.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10
ID,,,,,,,,,,
sub-1000021,7.625430,-110.843796,-31.410776,-14.304582,-58.193040,1.267348,-16.256983,-31.085651,-6.734729,-37.349705
sub-1000458,-2.686900,-99.696810,-16.900297,-11.260241,-49.261047,-40.481380,4.340034,-11.678871,-14.759934,-19.288872
sub-1000575,-3.391189,-41.532593,-16.283690,6.923420,-15.241653,-6.485703,4.359082,-1.681517,-10.869176,0.104506
sub-1000606,1.382976,-89.879420,-1.775408,18.353903,-51.242970,-12.489634,-2.822808,2.998624,22.957664,-6.521338
sub-1000963,11.553831,-154.482510,-12.179215,-20.130846,-81.294730,73.374030,3.920644,-44.567265,29.138690,-33.605183


### Scale your data

In [5]:
scaler = StandardScaler()
scaler.fit(embeddings_UKB)

scl_bdd_hcp = scaler.transform(embeddings_HCP)
scl_bdd_ukb = scaler.transform(embeddings_UKB)

### 2D UMAP

In [6]:
reducer2D = umap.UMAP(n_components=2, n_neighbors=200)

reducer2D.fit(scl_bdd_ukb)

bdd_2D_HCP = reducer2D.transform(scl_bdd_hcp)
bdd_2D_UKB = reducer2D.transform(scl_bdd_ukb)

/casa/home/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [7]:
bdd_2D_HCP = pd.DataFrame(bdd_2D_HCP, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])

bdd_2D_HCP['Dataset'] = 'hcp'
bdd_2D_UKB['Dataset'] = 'UkBioBank'

bdd_2D_HCP['ID'] = embeddings_HCP.index
bdd_2D_UKB['ID'] = embeddings_UKB.index

bdd_2D_All = pd.concat([bdd_2D_UKB,bdd_2D_HCP], axis=0)
bdd_2D_All.head()

,Dim 1,Dim 2,Dataset,ID
0,4.362270,4.831038,UkBioBank,sub-1000021
1,9.956990,7.940969,UkBioBank,sub-1000458
2,12.411469,4.796235,UkBioBank,sub-1000575
3,9.198896,3.964477,UkBioBank,sub-1000606
4,2.189068,4.121135,UkBioBank,sub-1000963


### Plot it in the notebook or write a html file

In [24]:
subject_id_list = []
dataset_name_list = []

In [21]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_All, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='2D UMAP HCP and UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ['UkBioBank', 'sub-1000575'],
                                   ...,
                                   ['UkBioBank', 'sub-6023847'],
                                   ['UkBioBank', 'sub-6024038'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '2f62b57b-fa09-45aa-b410-0e88a3f66156',
              'x': array([ 4.36227  ,  9.95699  , 12.411469 , ...,  6.584164 ,  8.870082 ,
                      

Clicked point ID: 869472, Dataset: hcp
Clicked point ID: 152225, Dataset: hcp
Clicked point ID: 784565, Dataset: hcp
Clicked point ID: 180533, Dataset: hcp
Clicked point ID: 870861, Dataset: hcp
Clicked point ID: 930449, Dataset: hcp
Clicked point ID: 127933, Dataset: hcp
Clicked point ID: 195647, Dataset: hcp
Clicked point ID: 169343, Dataset: hcp
Clicked point ID: 959574, Dataset: hcp
Clicked point ID: 729254, Dataset: hcp
Clicked point ID: 101309, Dataset: hcp
Clicked point ID: 792867, Dataset: hcp
Clicked point ID: 209834, Dataset: hcp
Clicked point ID: 884064, Dataset: hcp
Clicked point ID: 143527, Dataset: hcp
Clicked point ID: 138534, Dataset: hcp
Clicked point ID: 180230, Dataset: hcp
Clicked point ID: 759869, Dataset: hcp
Clicked point ID: 578057, Dataset: hcp
Clicked point ID: 138837, Dataset: hcp
Clicked point ID: 333330, Dataset: hcp
Clicked point ID: 164030, Dataset: hcp
Clicked point ID: 952863, Dataset: hcp


In [25]:
side = "R"
region = "CINGULATE"

bucket_files = []

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}buckets'

    if dataset.lower() in ['hcp']:
        path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset.lower()}/crops/2mm/{region}/mask/{side}buckets'

    filename = f'{path}/{subject_id}_cropped_skeleton.bck'#.minf'

    if os.path. isfile(filename):
        bucket_files.append(filename)
    else:
        print(f"{filename} is not a correct path, or the .bck doesn't exist")
bucket_files

['/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/138534_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/180230_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/759869_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/578057_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/138837_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/333330_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/164030_cropped_skeleton.bck',
 '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rbuckets/952863_cropped_skeleton.bck']

In [26]:
block = a.createWindowsBlock(4) # 4 columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = a.loadObject(file)
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])

no position could be read at 531, 221
no position could be read at 382, 264


In [12]:
from soma import aims

In [18]:
dic_vol = {}
dim = 0
rep = 0
while dim == 0 and rep < len(subject_id_list):
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset_name_list[rep]}/crops/2mm/{region}/mask/{side}crops"
    if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
        dim = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').np.shape
        sum_vol = np.zeros(shape=dim)
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
    rep += 1

for subject_id, dataset in zip(subject_id_list,dataset_name_list):
    if dataset.lower() in ['ukb', 'ukbiobank']:
        dataset = 'UkBioBank'
    elif dataset.lower() == 'hcp':
        dataset = 'hcp'
        
    mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

    if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
        vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        # compare the dim with the first file dim

        if vol.np.shape != dim:
            raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

            
        # to have a binary 3D structure
        dic_vol[subject_id] = (vol.np > 0).astype(int) 
        sum_vol += (vol.np > 0).astype(int) 
    else: 
        print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
        #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

sum_vol = sum_vol/len(subject_id_list)
#print(dic_vol[subject_id_list[0]].shape)
#print(np.count_nonzero(dic_vol[subject_id_list[0]]))

In [19]:
# Create axis
axes = list(dim[:3])
dim1 = list(dim[:3])[0]
dim2 = list(dim[:3])[1]
dim3 = list(dim[:3])[2]

# Create Data
data = sum_vol.reshape(list(dim[:3]))

X, Y, Z = np.mgrid[0:dim1:1, 0:dim2:1, 0:dim3:1]
values = np.flip(data, axis=[1,2])

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=values.flatten(),
    isomin=0.15,
    isomax=1,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.show()

In [26]:
"""
data_filenames_str = ' '.join(data_filenames_list)

#os.system(f'anatomist --input {data_filenames_str}')

# Command to run the Python script inside the container
command = f'bv bash -c "python visu_anatomist.py {data_filenames_str}"'

# Execute the command
os.system(command)
"""

'\ndata_filenames_str = \' \'.join(data_filenames_list)\n\n#os.system(f\'anatomist --input {data_filenames_str}\')\n\n# Command to run the Python script inside the container\ncommand = f\'bv bash -c "python visu_anatomist.py {data_filenames_str}"\'\n\n# Execute the command\nos.system(command)\n'

: 

In [31]:
# See for more information
# https://plotly.com/python/line-and-scatter/
# https://plotly.com/python/setting-graph-size/
# app.run_server(debug=True)